In [ ]:
import json
import pandas as pd
from IPython.display import HTML
from io import StringIO

pd.set_option("display.max_rows",500)

In [ ]:
def listify(my_list):
    """make a python list of strings into a html list (<ul>)
    :returns a <ul> as a string
    """
    ret = StringIO()
    ret.write("<ul>")
    for element in my_list:
        ret.write("<li>")
        ret.write(str(element))
        ret.write("</li>")
    ret.write("</ul>")
    return ret.getvalue()

# Read predicted captions from json

In [ ]:
def build_prediction_table(prediction_json, sampler):    
    with open(prediction_json) as captions_file:
        caption_json = json.load(captions_file)
    captions = pd.DataFrame(caption_json['sentences'])
    captions = captions.sort_values('video_id')
    captions['captions'] = captions['caption'].apply(lambda x: (sampler, x,))
    return captions

def add_thumbs(predition_table):
    predition_table['src'] = "test_frames/test_set_sampling_comparison/comparisons/"+predition_table['video_id']+"_concat.jpg"
    predition_table['img'] = '<img src="' + predition_table["src"] + '">'
    return predition_table

In [ ]:
pyscene = build_prediction_table('pyscene_test_predictions_epoch19.json','pyscene')
transnet = build_prediction_table('transnet_test_predictions_epoch19.json', 'transnet')

In [ ]:
transnet['video_id'] == pyscene['video_id']

In [ ]:
captions = pd.concat([transnet,pyscene])

In [ ]:
captions[captions['video_id'] == 'video7014']

In [ ]:
captions = captions.drop('caption',axis=1)

In [ ]:
captions = captions.groupby('video_id').agg({'captions':list})
captions = captions.reset_index()

In [ ]:
captions = add_thumbs(captions)

In [ ]:
captions['captions'] = captions['captions'].apply(
    lambda list_of_tuples: {tup[0]:tup[1] for tup in list_of_tuples}
)

def sort_caps_by_sample(sample_cap_dict):
    return [
        f"{sampler}:&nbsp;&nbsp;&nbsp;{sample_cap_dict.get(sampler)}"
        for sampler in ['random', 'pyscene', 'transnet']
    ]
captions['captions'] = captions['captions'].apply(sort_caps_by_sample)

In [ ]:
captions['captions'] = captions['captions'].apply(listify)

In [ ]:
captions

# Get human generated captions

In [ ]:
with open('test_videodatainfo.json','r') as f:
    expected_captions_json = json.load(f)

In [ ]:
expected_captions = pd.DataFrame(expected_captions_json['sentences'])

In [ ]:
expected_captions_json['videos']

In [ ]:
expected_captions = expected_captions.drop('sen_id',axis=1)

In [ ]:
expected_captions = expected_captions.groupby('video_id').agg({'caption':list})
expected_captions = expected_captions.rename(mapper={'caption': 'expected'},axis=1)

In [ ]:
expected_captions['expected_html'] = expected_captions['expected'].apply(listify)

In [ ]:
expected_captions

# Merge predicted & human captions

In [ ]:
captions = pd.merge(left=captions, right=expected_captions, on='video_id')
captions = captions.set_index('video_id')
captions = captions.drop(['src', 'expected'], axis=1)


In [ ]:
# Present Result

In [ ]:
table = HTML(captions[['expected_html','captions', 'img']].to_html(escape=False))

In [ ]:
# write to html file for easier viewing
with open('data_out.html','w') as out:
    out.write(table.data)


In [ ]:
# display in notebook
table